<a href="https://colab.research.google.com/github/grum47/Test_Mt5_Fractal/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D1%86%D0%B5%D0%BD_%D0%B8_%D0%BE%D0%B1%D1%8A%D0%B5%D0%BC%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Импорт нужных библиотек
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
%matplotlib inline
import numba as nb
from numba import jit


### 1) Подгрузить датафрейм из предоставленного файла data.feather.

In [ ]:
df = pd.read_feather('C:Downloads/data.feather')

### 2) Провести базовый EDA - посчитать среднее, медиану, стандартное отклонение, квартили для столбцов цен (_price_1) и объёмов (_vol_1) - отдельно для каждого столбца. Визуализировать боксплотами. Установить, были ли выбросы по ценам (например, через межквартильный размах).

In [ ]:
print(df.isnull().values.any())
df.head()

###### Переименовываем поля, для удобства работы

In [ ]:
df = df.rename(columns={
    'BTC-USD-SWAP@OKEX_bid_price_1' : 'bid_price',
    'BTC-USD-SWAP@OKEX_ask_price_1' : 'ask_price',
    'BTC-USD-SWAP@OKEX_bid_vol_1'   : 'bid_vol',
    'BTC-USD-SWAP@OKEX_ask_vol_1'   : 'ask_vol'
})
df.info()

###### Видим, что максимальное значение в столбцах bid_vol_1 и ask_vol_1 значительно превышает значения 3-го квартиля, что предполагает наличие выбросов

In [ ]:
df_des_tran = df.describe().T
df_des_tran

###### Строим боксплоты по Т датафрейму, для того, чтобы отрисовка была быстрее

In [ ]:
df_bplot = df_des_tran.drop(columns=['count', 'std', '50%']).T
boxplot = df_bplot.boxplot(column=['bid_price', 'ask_price']) 

###### Визуально видим выбросы

In [ ]:
boxplot = df_bplot.boxplot(column=['bid_vol', 'ask_vol']) 

###### Межквартильный размах. Видим, что есть точки превышающие значения 1го и 3го квартиля -+ 1,5 межквартильного размаха

In [ ]:
df_iqr = df_des_tran[['min', '25%', '75%', 'max', 'std']].copy()
df_iqr['iqr'] = df_iqr['75%'] - df_iqr['25%']
df_iqr['is_out'] = np.where((df_iqr['min'] < (df_iqr['25%'] - 1.5 * df_iqr['iqr'])) |
                            (df_iqr['max'] > (df_iqr['75%'] + 1.5 * df_iqr['iqr'])),
                             1,
                             0)
df_iqr

### 3) Визуализировать распределения цен и объёмов любым способом.

In [ ]:
plt.figure(figsize=(12, 6))
df['bid_price'].plot()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
df['ask_price'].plot()
plt.show()

###### Визуально также наблюдаем выбросы по объемам

In [ ]:
plt.figure(figsize=(12, 6))
df['bid_vol'].plot()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
df['ask_vol'].plot()
plt.show()

### 4) Рассчитать среднюю цену инструмента (bid_price+ask_price)/2 для всех точек и по ней рассчитать за для всех точек взвешенную по времени экспоненциальную скользящую среднюю по нижеописанному принципу. Замерить время выполнения (можно просто через %%timeit), попробовать достичь времени выполнения менее 1 с.
##### а. Инициализируемся средней ценой для первой строки датафрейма
##### б.Если разница по времени со следующей строчкой превышает 1 секунду, то условно считаем её равной одной секунде (параметр Δt_i в формуле ниже)
##### в.Очередное значение EMA = прошлое значение EMA + (очередная средняя цена - прошлое значение EMA)*(разница по времени до предыдущего значения)/(10000 секунд):
##### 𝐸𝑀𝐴𝑖=𝐸𝑀𝐴(𝑖−1)+(𝑀𝑖𝑑𝑃𝑟𝑖𝑐𝑒𝑖−𝐸𝑀𝐴(𝑖−1))(Δ𝑡𝑖)/(10000секунд)

In [ ]:
df_settlement = df.copy()

In [ ]:
%%timeit
df_settlement['avg_price'] = (df_settlement['bid_price'] + df_settlement['ask_price']) / 2
df_settlement['delta_t'] = df_settlement['exchange_time']\
                            .diff()\
                            .astype('timedelta64[ns]')\
                            .dt.total_seconds().\
                            fillna(0)
df_settlement['delta_t'] = np.where(df_settlement['delta_t'] >= 1,
                                    1,
                                    df_settlement['delta_t'])

a = np.array(df_settlement['avg_price'])
d = np.array(df_settlement['delta_t'])

@jit(nopython=True)
def ema(a, d):
    res = np.empty(len(d))
    res[0] = a[0]
    for i in range(1, len(res)):
        res[i] = res[i-1] + (a[i] - res[i-1]) * d[i] / 10000
    return res

df_settlement['ema'] = ema(a, d)

In [ ]:
df_settlement.head()

### 5) Ввести столбец position, инициализировать нулями

In [ ]:
df_settlement['position'] = 0

### 6) Рассчитать значения в этом столбце по следующей логике:
##### a) первые 10000 секунд - 0
##### b) далее, если средняя цена в очередной строке больше EMA более чем на 50 и значение в предыдущей строке pos меньше 10 -> +1
##### c) Если текущая средняя цена < EMA более чем на 50 и предыдущее значение pos > -10 -> -1
##### d) Иначе значение не меняется
### Постараться уложиться в 20 секунд.

In [ ]:
%%timeit
a = np.array(df_settlement['avg_price'])
d = np.array(
    (df_settlement['exchange_time'] - df_settlement['exchange_time'].iloc[0])\
    .dt.total_seconds()\
    .astype('int'))
e = np.array(df_settlement['ema'])
p = np.array(df_settlement['position'])


@jit(nopython=True)
def pos(a, d, e, p):
    position = np.empty(len(p))
    position[0] = p[0]
    for i in range(1, len(position)):
        if d[i] <= 10000:
            position[i] = 0
        else:
            if (a[i] > (e[i] + 50)) and (position[i-1] < 10):
                position[i] = position[i-1] + 1
            elif (a[i] < (e[i] - 50)) and (position[i-1] > -10):
                position[i] = position[i-1] - 1
            else:
                position[i] = p[i]
    return position


df_settlement['position'] = pos(a, d, e, p)

In [ ]:
df_settlement

### 7) Визуализировать результаты.

In [ ]:
plt.figure(figsize=(12, 6))
df_settlement['ema'].plot()
plt.show()

In [ ]:
plot = df_settlement.plot.scatter(x='avg_price', y='ema', c='position')
plt.show()